# utils

> Fill in a module description here

In [ ]:
#|default_exp utils

In [ ]:
%env XLA_PYTHON_CLIENT_MEM_FRACTION=0.2

env: XLA_PYTHON_CLIENT_MEM_FRACTION=0.2


In [ ]:
#|export
from collections import defaultdict

import yaml
import jax
import jax.numpy as jnp
import numpy as np

In [ ]:
jax.default_backend()

'gpu'

From `lovely-numpy`: https://github.com/xl0/lovely-numpy/blob/master/nbs/03_utils.utils.ipynb

In [ ]:
#|export
def sci_mode(f: float):
    return ((abs(f) < 10**-4) or
            (abs(f) > 10**3))

def pretty_str(x):
    """A slightly better way to print `float`-y values.
    Works for `np.ndarray`, `torch.Tensor`, `jax.DeviceArray`, and scalars."""

    if isinstance(x, int):
        return '{}'.format(x)
    elif isinstance(x, float):
        if x == 0.:
            return "0."

        sci = sci_mode(x)
        fmt = f"{{:.3{'e' if sci else 'f'}}}"

        return fmt.format(x)
    elif x.ndim == 0:
            return pretty_str(x.item())
    else:
        slices = [pretty_str(x[i]) for i in range(0, x.shape[0])]
        return '[' + ", ".join(slices) + ']'

def sparse_join(lst, sep=" "):
    # Join non-empty list elements into a space-separated string
    return sep.join( [ l for l in lst if l] )

def ansi_color(s: str, col: str, use_color=True):
        "Very minimal ANSI color support"
        style = defaultdict(str)
        style["grey"] = "\x1b[38;2;127;127;127m"
        style["red"] = "\x1b[31m"
        end_style = "\x1b[0m"
        return style[col]+s+end_style if use_color else s

In [ ]:
# |export
def to_str(
    x,              # Input
    color=True ,    # ANSI color highlighting  
    ddof=0          # For "std" unbiasing
    ):
    x = np.array(x)  # in jnp the code breaks, see https://github.com/google/jax/issues/14721
    if x.size == 0:
        return ansi_color("empty", "grey", color)

    zeros = ansi_color("all_zeros", "grey", color) if np.equal(x, 0.0).all() and x.size > 1 else None
    pinf = ansi_color("+Inf!", "red", color) if np.isposinf(x).any() else None
    ninf = ansi_color("-Inf!", "red", color) if np.isneginf(x).any() else None
    nan = ansi_color("NaN!", "red", color) if np.isnan(x).any() else None

    attention = sparse_join([zeros, pinf, ninf, nan])
    summary = None
    if not zeros:
        # Calculate stats on good values only.
        gx = x[np.isfinite(x)]
        minmax = f"x∈[{pretty_str(gx.min())}, {pretty_str(gx.max())}]" if gx.size > 2 else None
        meanstd = f"μ={pretty_str(gx.mean())} σ={pretty_str(gx.std(ddof=ddof))}" if gx.size >= 2 else None
        summary = sparse_join([minmax, meanstd])

    return sparse_join([summary, attention])


In [ ]:
to_str(jnp.array([[0.],[0.]]))

'\x1ball_zeros\x1b'

In [ ]:
to_str(jnp.array([[1.],[4.],[3.]]))

'x∈[1.000, 4.000] μ=2.667 σ=1.247'

In [ ]:
#|export
def str_tree(tree):
    leaves, treedef = jax.tree_util.tree_flatten(tree)
    strs = [to_str(l, color=False) for l in leaves]
    ptree = jax.tree_util.tree_unflatten(treedef, strs)
    return yaml.dump(ptree, indent=2, allow_unicode=True)

In [ ]:
#|hide
import nbdev;
nbdev.nbdev_export()